[View in Colaboratory](https://colab.research.google.com/github/raviteja8484/ML/blob/master/4Baccuracy87(8).ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 12
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 64, dropout_rate = 0.3):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 64, dropout_rate = 0.3):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 64
dropout_rate = 0.3
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [17]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_105 (Conv2D)             (None, 32, 32, 64)   1728        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_105 (BatchN (None, 32, 32, 64)   256         conv2d_105[0][0]                 
__________________________________________________________________________________________________
activation_105 (Activation)     (None, 32, 32, 64)   0           batch_normalization_105[0][0]    
__________________________________________________________________________________________________
conv2d_106

Trainable params: 3,069,450
Non-trainable params: 24,128
__________________________________________________________________________________________________


In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 520s 10ms/step - loss: 1.4869 - acc: 0.4544 - val_loss: 2.2372 - val_acc: 0.4011

Epoch 00001: val_acc improved from -inf to 0.40110, saving model to weights.best.hdf5
Epoch 2/50
14464/50000 [=======>......................] - ETA: 5:27 - loss: 1.1241 - acc: 0.5921

50000/50000 [==============================] - 495s 10ms/step - loss: 1.0339 - acc: 0.6291 - val_loss: 1.5964 - val_acc: 0.5368

Epoch 00002: val_acc improved from 0.40110 to 0.53680, saving model to weights.best.hdf5
Epoch 3/50
30848/50000 [=================>............] - ETA: 2:55 - loss: 0.8650 - acc: 0.6888

50000/50000 [==============================] - 495s 10ms/step - loss: 0.8388 - acc: 0.6999 - val_loss: 1.8386 - val_acc: 0.5529

Epoch 00003: val_acc improved from 0.53680 to 0.55290, saving model to weights.best.hdf5
Epoch 4/50
35328/50000 [====================>.........] - ETA: 2:14 - loss: 0.7275 - acc: 0.7417

50000/50000 [==============================] - 495s 10ms/step - loss: 0.7147 - acc: 0.7468 - val_loss: 1.7029 - val_acc: 0.5800

Epoch 00004: val_acc improved from 0.55290 to 0.58000, saving model to weights.best.hdf5
Epoch 5/50
36480/50000 [====================>.........] - ETA: 2:04 - loss: 0.6331 - acc: 0.7759

50000/50000 [==============================] - 494s 10ms/step - loss: 0.6304 - acc: 0.7778 - val_loss: 0.9895 - val_acc: 0.7130

Epoch 00005: val_acc improved from 0.58000 to 0.71300, saving model to weights.best.hdf5
Epoch 6/50
36864/50000 [=====================>........] - ETA: 2:00 - loss: 0.5638 - acc: 0.8038

...

50000/50000 [==============================] - 498s 10ms/step - loss: 0.1397 - acc: 0.9495 - val_loss: 0.5931 - val_acc: 0.8644

Epoch 00032: val_acc improved from 0.85710 to 0.86440, saving model to weights.best.hdf5
Epoch 33/50
38784/50000 [======================>.......] - ETA: 1:43 - loss: 0.1327 - acc: 0.9525

50000/50000 [==============================] - 498s 10ms/step - loss: 0.1334 - acc: 0.9521 - val_loss: 0.7915 - val_acc: 0.8432

Epoch 00033: val_acc did not improve from 0.86440
Epoch 34/50
42752/50000 [========================>.....] - ETA: 1:06 - loss: 0.1317 - acc: 0.9539

50000/50000 [==============================] - 498s 10ms/step - loss: 0.1320 - acc: 0.9537 - val_loss: 0.6998 - val_acc: 0.8424

Epoch 00034: val_acc did not improve from 0.86440
Epoch 35/50
43904/50000 [=========================>....] - ETA: 56s - loss: 0.1197 - acc: 0.9562

50000/50000 [==============================] - 498s 10ms/step - loss: 0.1213 - acc: 0.9555 - val_loss: 0.5730 - val_acc: 0.8783

Epoch 00035: val_acc improved from 0.86440 to 0.87830, saving model to weights.best.hdf5
Epoch 36/50
38656/50000 [======================>.......] - ETA: 1:44 - loss: 0.1163 - acc: 0.9572

40192/50000 [=======================>......] - ETA: 1:30 - loss: 0.1178 - acc: 0.9565

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 0.7949684534072876
Test accuracy: 0.7609


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')